In [ ]:
import numpy as np
import librosa
import os
from matplotlib import pyplot as plt
!pip install pretty_midi
import pretty_midi 
pretty_midi.pretty_midi.MAX_TICK = 1e10

     |████████████████████████████████| 5.6 MB 6.1 MB/s 
     |████████████████████████████████| 51 kB 8.1 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=b455b88ca06e8621897c889c7d6c5e8dee9d50aa999da7d9e94c7247996b0072
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi


In [ ]:
#preparing the data from the original audio files
entries = os.scandir('/content/drive/MyDrive/train_source')#loading the data to process which we are going to use for training 

for file in entries:
    file_name,file_extensions=os.path.splitext(file)

    if file_extensions=='.wav':
        y, sr = librosa.load(file, sr=16000)
        n_fft=2048
        hop_length=512
        stft=librosa.core.stft(y, n_fft=n_fft, hop_length=hop_length)#using the librosa library to calculate the stft of the audio file which is returned as a 2D vector 
        c=len(y) # it is being stored because it will be used for the frequency while making the piano roll file
        stft=np.abs(stft)# the returned 2d vector is a complex quantity 
        stft=librosa.amplitude_to_db(stft)# taking the amplitude on logarithmic scale gives a better visualization of data
        b=np.shape(stft)[1]
        print(np.shape(np.transpose(stft)))
        entries1 = os.scandir('/content/drive/MyDrive/train_source')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used
        for file1 in entries1:
          file_name1,file_extensions1=os.path.splitext(file1)
        
          if file_name1==file_name and file_extensions1==".mid":
            file2=file_name + file_extensions1
            pm = pretty_midi.PrettyMIDI(file2)
            a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The Sampling frequency has been taken such that its shape and the stft files shape is same 
            a = (a[21:109, :b] > 0)#we only need the frequency of the piano and converting non binary data to 0 and 1
            a=np.transpose(a)
            print(np.shape(a))
            out = '/content/drive/MyDrive/trainstft/stftmid/mid_'+str(file_name1[40:])+'mid'+'.npy'
            np.save(out,a)
            out = '/content/drive/MyDrive/trainstft/stftwav/wav_'+str(file_name[40:])+'.npy'
            stft=np.transpose(stft)
            if a.shape[0]<stft.shape[0]: #cutting out little extra part in mid arrays
              stft=stft[:a.shape[0],:]
            print(np.shape(stft))  
            np.save(out,stft)
            print('wrote file', out)
            print("saving", out)

In [ ]:
path1='/content/drive/MyDrive/trainstft/stftwav'

def concatenate_wav(path):#we are dividing the number of features of the stft file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,1025))#initialising the array with the correct shape 

  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]# adding a new axis to the original files
      for b in range(int(np.shape(a)[1]/100)):
          
          c=a[:,b*100:(b*100)+100,:]# dividing them into batches of 100
          d=np.concatenate((d,c), axis=0)# stacking them up on the first axis 
      print(np.shape(d)) 
  out = '/content/drive/MyDrive/trainstft/stftwav/concatenate'+'wav'+'.npy'
  d=d[1:,:,:]# array was initialise with zeros so excluding that part 

  np.save(out,d)  
  print(np.shape(d))

concatenate_wav(path1)

path='/content/drive/MyDrive/trainstft/stftmid'

def concatenate_mid(path):#we are dividing the number of features of the pianoroll file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,88))
  l=0
  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]
      for b in range(int(np.shape(a)[1]/100)):
            
          c=a[:,b*100:(b*100)+100,:]
          d=np.concatenate((d,c), axis=0)
      print(np.shape(d))      

          
  out = '/content/drive/MyDrive/trainstft/stftmid/concatenate'+'mid'+'.npy'
  d=d[1:,:,:]
  np.save(out,d)  

concatenate_mid(path)


In [ ]:
#preparing the data from the original audio files
entries = os.scandir('/content/drive/MyDrive/test_source')#loading the data to process which we are going to use for training 

for file in entries:
    file_name,file_extensions=os.path.splitext(file)

    if file_extensions=='.wav':
        y, sr = librosa.load(file)
        n_fft=2048
        hop_length=512
        stft=librosa.core.stft(y, n_fft=n_fft, hop_length=hop_length)#using the librosa library to calculate the stft of the audio file which is returned as a 2D vector 
        c=len(y) # it is being stored because it will be used for the frequency while making the piano roll file
        stft=np.abs(stft)# the returned 2d vector is a complex quantity 
        stft=librosa.amplitude_to_db(stft)# taking the amplitude on logarithmic scale gives a better visualization of data
        b=np.shape(stft)[1]
        print(np.shape(np.transpose(stft)))
        entries1 = os.scandir('/content/drive/MyDrive/test_source')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used
        for file1 in entries1:
          file_name1,file_extensions1=os.path.splitext(file1)

          if file_name1==file_name and file_extensions1==".mid":
            file2=file_name + file_extensions1
            pm = pretty_midi.PrettyMIDI(file2)
            a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The Sampling frequency has been taken such that its shape and the stft files shape is same 
            a = (a[21:109, :b] > 0)#we only need the frequency of the piano and converting non binary data to 0 and 1
            a=np.transpose(a)
            print(np.shape(a))
            out = '/content/drive/MyDrive/teststft/stftmid/mid_'+str(file_name1[40:])+'mid'+'.npy'
            np.save(out,a)
            out = '/content/drive/MyDrive/teststft/stftwav/wav_'+str(file_name[40:])+'.npy'
            stft=np.transpose(stft)
            if a.shape[0]<stft.shape[0]: #cutting out little extra part in mid arrays
              stft=stft[:a.shape[0],:]
            print(np.shape(stft))  
            np.save(out,stft)
            print('wrote file', out)
            print("saving", out)

  
  

In [ ]:
path1='/content/drive/MyDrive/teststft/stftwav'

def concatenate_wav(path):#we are dividing the number of features of the stft file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,1025))#initialising the array with the correct shape 

  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]# adding a new axis to the original files
      for b in range(int(np.shape(a)[1]/100)):
          
          c=a[:,b*100:(b*100)+100,:]# dividing them into batches of 100
          d=np.concatenate((d,c), axis=0)# stacking them up on the first axis 
      print(np.shape(d)) 
  out = '/content/drive/MyDrive/teststft/stftwav/concatenate'+'wav'+'.npy'
  d=d[1:,:,:]# array was initialise with zeros so excluding that part 

  np.save(out,d)  
  print(np.shape(d))

concatenate_wav(path1)

path='/content/drive/MyDrive/teststft/stftmid'

def concatenate_mid(path):#we are dividing the number of features of the pianoroll file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,88))
  l=0
  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]
      for b in range(int(np.shape(a)[1]/100)):
            
          c=a[:,b*100:(b*100)+100,:]
          d=np.concatenate((d,c), axis=0)
      print(np.shape(d))      

          
  out = '/content/drive/MyDrive/teststft/stftmid/concatenate'+'mid'+'.npy'
  d=d[1:,:,:]
  np.save(out,d)  

concatenate_mid(path)
